### Imports utilizados

In [27]:
import requests
import time
import random
import re
import pandas as pd
import os
from tqdm import tqdm as tqdm
from bs4 import BeautifulSoup

### Parsing da Página Principal da Wikipedia

In [11]:
a = requests.get('https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal')
soup = BeautifulSoup(a.text, 'html.parser')

Para cada link na seção "main" da página principal, verifica se o link representa um verbete, caso positivo, adiciona à um Set para evitar duplicatas

In [12]:
links = soup.main.find_all('a')
all_verbets = set()
for link in links:
    if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])

Escolhe aleatoriamente um verbet da conjunto obtido da página principal, e faz o mesmo processamento. O loop é executado até que o conjunto de verbetes tenha 5000 verbetes.

In [13]:
while(len(all_verbets) < 500):
    verbet = random.choice(list(all_verbets))
    a = requests.get('https://pt.wikipedia.org' + verbet)
    soup = BeautifulSoup(a.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        if link.has_attr('href') and re.match(r'/wiki/.*', link['href']) and not re.match(r'/wiki/.*:.*', link['href']):
            all_verbets.add(link['href'])
    time.sleep(random.randint(1,2))

Cria o diretório para armazenar os verbetes, percorre todo o conjunto e salva o HTML

In [14]:
os.makedirs('verbetes', exist_ok=True)
for verbet in tqdm(all_verbets):
    titulo = verbet.split('/')[-1]
    soup = BeautifulSoup(a.text, 'html.parser')
    with open(f'verbetes/{titulo}.html', 'w') as f:
        f.write(soup.prettify())
    time.sleep(random.randint(1,2))

  0%|          | 2/1063 [00:04<35:24,  2.00s/it]


KeyboardInterrupt: 

In [31]:
e = requests.get("https://pt.wikipedia.org/wiki/Alan_Turing")
soup = BeautifulSoup(e.text, 'html.parser') 
tables = soup.findAll('table')
infobox = ""
for table in tables:
    if "infobox" in table["class"] or "infobox_v2" in table["class"]:
        infobox = table
        break

chava = {}

titulo = infobox.findAll("th")[0]
for tr in soup.findAll('tr'):
    if len(tr.findAll("td")) == 2:
        messi = tr.findAll("td")
        chava[messi[0].text.strip()] = [li.text.strip() for li in messi[1].findAll("li")] if messi[1].findAll("li") else messi[1].text.strip()
        # chava[messi[0]] = messi[1].text
        # print(messi[0].text, messi[1].text)
        
print(chava)
chava_df = pd.DataFrame.from_dict(chava, orient='index')
chava_df.transpose().to_json('chava_transpose.json', orient='index')
print(chava_df.transpose())

        

{'Nome completo': 'Alan Mathison Turing', 'Conhecido(a) por': ['Criptoanálise da Máquina Enigma', 'Problema da parada', 'Máquina de Turing', 'Teste de Turing', 'Máquinas desorganizadas', 'Modelo Turing', 'Redução de Turing', 'Base química da morfogênese'], 'Nascimento': '23 de junho de 1912Maida Vale, Londres', 'Morte': '7 de junho de 1954\xa0(41\xa0anos)Wilmslow, Cheshire', 'Causa da morte': 'Suicídio por ingestão de cianeto', 'Nacionalidade': 'britânico', 'Educação': 'Sherborne School', 'Alma mater': ['Universidade de Cambridge (BA, MA)', 'Universidade de Princeton (PhD)'], 'Prêmios': 'Prêmio Smith (1936)', 'Religião': 'Ateísmo', 'Orientador(es)(as)': 'Alonzo Church', 'Orientado(a)(s)': ['Robin Gandy', 'Beatrice Worsley'], 'Instituições': ['Universidade de Manchester', 'Escola Governamental de Código e Cifra', 'Laboratório Nacional de Física'], 'Campo(s)': ['Lógica', 'matemática', 'criptoanálise', 'ciência da computação', 'biologia matemática e teórica'], 'Tese': 'Sistemas de Lógica 